In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import math
from datetime import datetime,timedelta
import time

In [ ]:
def get_newscontent(l_url) :
    news_content = []
    text_save = []
    logi_url = 'http://www.klnews.co.kr/news/'+l_url
    print(logi_url)
    get_req = requests.get(logi_url)
    get_soup = BeautifulSoup(get_req.content, 'html.parser')
    
    try:
        sub_title = get_soup.select('td.subartView-sub-title')[0].text.replace('“' , '').replace('”','') #부제목
    except:
        sub_title =''
    news_content.append(sub_title)
        

    
    
    try:
        _text = get_soup.select('div >p')
        for i in range(0,len(_text)-1) : #내용저장
            text_save.append(get_soup.select('div >p')[i].text)
        text_save = ''.join(text_save)
    except :
        text_save =''
    news_content.append(text_save)
    
    return news_content
    
    
    


In [ ]:
page = 1
url = 'http://www.klnews.co.kr/news/articleList.html?page='+str(page)+'&total=57252&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type='
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
maxpage = soup.select('font')[1].text
maxpage= math.ceil(int(maxpage[6:-2].replace(',',"").strip())/20)

news_box =[]
print('총페이지',maxpage)

while page < maxpage +1 :
    print('현재페에지', page)
    a = 0
    for i in range(len(soup.select('td.ArtList_Title > a'))):
        
        try :
            
            urls = soup.select('td.ArtList_Title > a')[i]['href']
            news_content2 = get_newscontent(urls)
            data = soup.select('td.View_SmFont.FontEng')[i].text
            classify = soup.select('td.ArtList_Title > font')[i].text.replace('[',"").replace(']',"")
            title = soup.select('td.ArtList_Title > a')[i].text
            sub_title= news_content2[0]
            contents = news_content2[1]
            news_total = [data , classify, title,sub_title,contents]
            news_box.append(news_total)
                
        
        except Exception as e:
            print(e)
        
            
            
    page +=1
    time.sleep(2)
    url = 'http://www.klnews.co.kr/news/articleList.html?page='+str(page)+'&total=57252&sc_section_code=&sc_sub_section_code=&sc_serial_code=&sc_area=&sc_level=&sc_article_type=&sc_view_level=&sc_sdate=&sc_edate=&sc_serial_number=&sc_word=&sc_word2=&sc_andor=&sc_order_by=E&view_type='
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    

In [ ]:
data =pd.DataFrame(news_box)
data.columns = ['data', 'class', 'title','subtitle', 'content']
data.to_excel('./loginews.xlsx', index =False)